In [1]:
#These lines set up inline plotting, and apply a standard size
%matplotlib inline
import matplotlib
matplotlib.rc('font', **{'size': 22})
# Standard includes
import datetime
import re
import os
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import matplotlib.pyplot as plt
from time import time
import timeit
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from matplotlib.dates import HourLocator

In [2]:
quantities = [
    'attemptnr'
]

transforms = {'Reco_tf.py':{},
              'DAODMerge_tf.py':{},
              'PRWConfig_tf.py':{},
              'NTUPMerge_tf.py':{}
             }

In [3]:
# Define query for the tasks
jobQuery = {
    "query": {
        "bool": {
            "should": [
                {"term":{"homepackage": "AthDerivation/21.2.8.0"}}
                #{"term":{"homepackage": "AthDerivation/21.2.9.0"}}
                #{"term":{"homepackage": "AthDerivation/21.2.10.0"}}
            ]
        }
        "wildcard": {
        "destinationdblock": "*data*"
        }
    }
}

In [5]:
# Execute the queries
es = Elasticsearch(['atlas-kibana-dev.mwt2.org'],timeout=120)

# Tasks
jobIndex = "jobs_archive_2018*,jobs_archive_2017*"
jobs = scan(es, query=jobQuery, index=jobIndex, scroll='5m', timeout="5m", size=1000)

#matchingTasks = sum(1 for x in jobs)

#print("Found %i matching jobs" %matchingTasks)

In [6]:
start_time = timeit.default_timer()
jobsCounter = 0
matchingTasks = 10000000
for res in jobs: # Loop over jobs
    #print(jobsCounter)
    jobsCounter += 1
    if jobsCounter % 100000 == 0: 
        et = timeit.default_timer() - start_time
        print("Processing task ",jobsCounter," Elapsed Time:", et, '\t projected finish in:',et/jobsCounter*matchingTasks - et )
    transf = res['_source']['transformation']
    if not transf in transforms:
        print("WARNING \t Transform %s unknown"%transf)
        continue
    for quantity in quantities: # get the relevant quantities
        if not quantity in transforms[transf].keys(): 
            transforms[transf][quantity] = [res['_source'][quantity]]
        else:
            transforms[transf][quantity].append(res['_source'][quantity])

Processing task  100000  Elapsed Time: 24.827092200517654 	 projected finish in: 2457.882127851248
Processing task  200000  Elapsed Time: 46.10969212651253 	 projected finish in: 2259.374914199114
Processing task  300000  Elapsed Time: 67.03517631068826 	 projected finish in: 2167.470700712254
Processing task  400000  Elapsed Time: 89.34349635057151 	 projected finish in: 2144.2439124137163
Processing task  500000  Elapsed Time: 111.47735922411084 	 projected finish in: 2118.069825258106
Processing task  600000  Elapsed Time: 134.55113931372762 	 projected finish in: 2107.9678492483995
Processing task  700000  Elapsed Time: 158.25484473444521 	 projected finish in: 2102.528651471915
Processing task  800000  Elapsed Time: 177.77700069732964 	 projected finish in: 2044.4355080192909
Processing task  900000  Elapsed Time: 199.06331851892173 	 projected finish in: 2012.7513316913196
Processing task  1000000  Elapsed Time: 221.2817970737815 	 projected finish in: 1991.5361736640334
Processi

KeyboardInterrupt: 

In [ ]:
transforms.keys()
transforms['DAODMerge_tf.py']
transforms

In [ ]:
len(jobs)